# Non-Linear Poisson in 2D

In this section, we consider the non-linear Poisson problem:

$$
-\nabla \cdot \left( (1+u^2) \nabla u \right) = f, \Omega
\\
u = 0, \partial \Omega
$$
where $\Omega$ denotes the unit square.

For testing, we shall take a function $u$ that fulfills the boundary condition, the compute $f$ as

$$
f(x,y) = -\nabla^2 u - F(u)
$$

We will use the Picard Method, which reads in the variational formulation

$$
\mbox{Find } u_{n+1} \in \mathcal{V}_h, \mbox{such that}\\
\int_{\Omega} (1+u_n^2) \nabla u_{n+1} \cdot \nabla v ~ d\Omega = \int_{\Omega} f v ~d\Omega, \quad \forall v \in \mathcal{V}_h
$$

In [1]:
from functools import partial
from pyccel.epyccel import epyccel
from simplines import assemble_vector, assemble_matrix, assemble_scalar, apply_dirichlet

### Example 1

In this example, we take $u(x,y) = \sin(\pi x) \sin(\pi y)$

In [2]:
from gallery_section_08 import assemble_vector_ex01, assemble_matrix_ex01, assemble_norm_ex01

In [3]:
assemble_matrix_ex01_pyccel = epyccel(assemble_matrix_ex01)
assemble_stiffness = partial(assemble_matrix, assemble_matrix_ex01_pyccel)

In [4]:
#assemble_vector_ex01_pyccel = epyccel(assemble_vector_ex01)
#assemble_rhs = partial(assemble_vector, assemble_vector_ex01_pyccel)

assemble_rhs = partial(assemble_vector, assemble_vector_ex01)

In [5]:
#assemble_norm_ex01_pyccel = epyccel(assemble_norm_ex01)
#assemble_norm_l2 = partial(assemble_scalar, assemble_norm_ex01_pyccel)

assemble_norm_l2 = partial(assemble_scalar, assemble_norm_ex01)

In [6]:
from simplines import SplineSpace
from simplines import TensorSpace
from simplines import StencilMatrix
from simplines import StencilVector

# create the spline space for each direction
V1 = SplineSpace(degree=2, nelements=16)
V2 = SplineSpace(degree=2, nelements=16)

# create the tensor space
V = TensorSpace(V1, V2)

In [8]:
niter = 10

rhs = StencilVector(V.vector_space)
u   = StencilVector(V.vector_space)

for i in range(niter):
    stiffness = assemble_stiffness(V, fields=[u])
    stiffness = apply_dirichlet(V, stiffness)

    M = stiffness.tosparse() 

    from scipy.sparse import csc_matrix, linalg as sla

    lu = sla.splu(csc_matrix(M))    
        
    rhs = assemble_rhs( V )
    rhs = apply_dirichlet(V, rhs)
    
    b = rhs.toarray()
    
    x = lu.solve(b)
    x = x.reshape(V.nbasis)
    
    u.from_array(V, x)
    u = apply_dirichlet(V, u)
    
    l2_norm = assemble_norm_l2(V, fields=[u])
    print('> l2-norm = ', l2_norm)    

> l2-norm =  0.10416232003536087
> l2-norm =  0.019794500321175343
> l2-norm =  0.003272950864084159
> l2-norm =  0.0004336036255791072
> l2-norm =  5.427312013037896e-05
> l2-norm =  2.6656604628473587e-05
> l2-norm =  2.611607929433604e-05
> l2-norm =  2.612392518145497e-05
> l2-norm =  2.612312922837201e-05
> l2-norm =  2.6123179836958155e-05
